In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict

import requests
from bs4 import BeautifulSoup
import re
import os
import time
import csv

import matplotlib.pyplot as plt
%matplotlib inline

/Users/scott/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/scott/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Make a data folder if does not already exist
import os
if not os.path.exists('data/'):
    os.makedirs('data/')

# Determine IDs of articles to scrape
ELife since 2014

In [3]:
# Load database of available articles
df = pd.read_csv('/gh/data/opencode/PMC-ids.csv')

/Users/scott/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_keep = df[df['Journal Title']=='eLife']
df_keep = df_keep[df_keep['Year']>=2014]
df_keep.reset_index(inplace=True, drop=True)

In [5]:
df_keep.head()

,Journal Title,ISSN,eISSN,Year,Volume,Issue,Page,DOI,PMCID,PMID,Manuscript Id,Release Date
0,eLife,NaN,2050-084X,2014,3,NaN,e01539,10.7554/eLife.01539,PMC3881093,24399458.0,NaN,live
1,eLife,NaN,2050-084X,2014,3,NaN,e01386,10.7554/eLife.01386,PMC3882429,24399457.0,NaN,live
2,eLife,NaN,2050-084X,2014,3,NaN,e01928,10.7554/eLife.01928,PMC3882917,24399459.0,NaN,live
3,eLife,NaN,2050-084X,2014,3,NaN,e01385,10.7554/eLife.01385,PMC3884117,24424411.0,NaN,live
4,eLife,NaN,2050-084X,2014,3,NaN,e01457,10.7554/eLife.01457,PMC3885792,24424413.0,NaN,live


# Scrape all

In [6]:
apikey = open('apikey.txt', 'r').read()
db = 'pmc'
base = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?'

In [7]:
# Load keywords
with open('keywords.csv', 'r') as f:
    reader = csv.reader(f)
    terms = list(reader)[0]

In [ ]:
# Number of characters to extract around keywords
span_buffer = 100

In [10]:
# Saving parameters
N_previous = 600
N_chunk = 10000
N_save = 100

In [12]:
# Load previous computation
if N_previous > 0:
    dfs_articles = {}
    for k in terms:
        csv_name = 'data/{:s}_{:d}.csv'.format(k, N_previous)
        dfs_articles[k] = [pd.read_csv(csv_name, index_col=0)]
else:
    dfs_articles = defaultdict(list)

for i, row in df_keep.loc[N_previous+1:].iterrows():
    # Get full text of 1 paper
    pmcid = row['PMCID']
    s = '{:s}db={:s}&id={:s}'.format(base, db, pmcid, apikey)
    out = requests.get(s)
    bs = BeautifulSoup(out.content, 'lxml')

    # DFs of terms
    for term in terms:
        dict_term = defaultdict(list)
        for s in re.finditer(term, out.text, re.IGNORECASE):
            save_span = s.span()
            sent = out.text[(save_span[0] - span_buffer):(save_span[1] + span_buffer)]

            dict_term['PMCID'].append(pmcid)
            dict_term['sentence'].append(sent)
        dfs_articles[term].append(pd.DataFrame(dict_term))

    # Save output every N
    if (i % N_save == 0) & (i > 0):
        print(i)
        for k in dfs_articles.keys():
            df_save = pd.concat(dfs_articles[k])
            df_save.to_csv('data/{:s}_{:d}.csv'.format(k, i))

            # Delete last file Unless 
            if (i-N_save) % N_chunk > 0:
                os.remove('data/{:s}_{:d}.csv'.format(k, i - N_save))

        if i % N_chunk == 0:
            if i > 0:
                dfs_articles = defaultdict(list)
            
# Save output when finish
for k in dfs_articles.keys():
    df_save = pd.concat(dfs_articles[k])
    df_save.to_csv('data/{:s}_{:d}.csv'.format(k, i))
    os.remove('data/{:s}_{:d}.csv'.format(k, int(np.round(i-(N_save/2-1), -2))))

700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800


FileNotFoundError: [Errno 2] No such file or directory: 'data/data shared_4850.csv'